# **Prediction on Test Set**

I am re-using the code from Training notebook and not changing any variable names for consistency and ensuring that all transformations are done in the same way as they were done in the test set.

So, variables will have names like train, train_inp in this notebook. They are all test ones, Apologies for the inconvinience.


In [ ]:
#importing basic libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

**The purpose of this Notebook is using the best model trained in the training notebook to predict on test set. The reason most of transformation done to the train data are done to the test data by using repeated code is to maintain the same transformations and also as lot of changes were made**

In [ ]:
#importing test dataset

train_inp=pd.read_csv('/Test_Inpatientdata-1542969243754.csv')
train_out=pd.read_csv('/Test_Outpatientdata-1542969243754.csv')
train_ben=pd.read_csv('/Test_Beneficiarydata-1542969243754.csv')
train_lab=pd.read_csv('/Test-1542969243754.csv')

In [ ]:
print(train_inp.columns)
print(train_inp.shape)

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')
(9551, 30)


In [ ]:
print(train_out.columns)
print(train_out.shape)

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
      dtype='object')
(125841, 27)


In [ ]:
print(train_ben.columns)
print(train_ben.shape)

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')
(63968, 25)


In [ ]:
print(train_lab.columns)
print(train_lab.shape)

Index(['Provider'], dtype='object')
(1353, 1)


# **Preprocessing the Dataset to make it similar to Training Set**

**We have 4 dataframes for train dataset which have a lot of common columns, So let's merge them in order to make predictions on test set.**

In [ ]:
# adding a feature named inpatient: 1=yes, 0=no

train_inp['Hospitalized'] = 1
train_out['Hospitalized'] = 0

In [ ]:
# merging the train dataframes

common_cols=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode', 'Hospitalized']

train_merged = pd.merge(train_out, train_inp, left_on=common_cols,
                        right_on=common_cols, how='outer')

train_merged=pd.merge(train_merged, train_ben, on="BeneID")

train=pd.merge(train_merged, train_lab, on="Provider")



**Adding meaningful feature out of claim start and claim end date**

In [ ]:
# adding claim period feature

startdate= pd.to_datetime( train["ClaimStartDt"] )
enddate= pd.to_datetime( train["ClaimEndDt"] )

period = (enddate - startdate).dt.days
train["Claim_Period"] = period

In [ ]:
# adding a meaningful feature out of admit date(hospitalization) and release date

startdate= pd.to_datetime( train["AdmissionDt"] )
enddate= pd.to_datetime( train["DischargeDt"] )

period = ( enddate - startdate).dt.days
train["Admit_Period"] = period

In [ ]:
# now we can drop the four dates as they are not needed

train=train.drop(columns=["ClaimStartDt" , "ClaimEndDt", "AdmissionDt", "DischargeDt"], axis=1)


In [ ]:
# Extract columns with names starting with "ChronicCond"

chronic_cols_names = train.columns[train.columns.str.startswith("ChronicCond")]

# Replace values of 2 with 0 in the extracted columns

train[chronic_cols_names] = train[chronic_cols_names].replace({2: 0})

In [ ]:
# Replace values of 2 with 0 in the "Gender" column

train["Gender"] = train["Gender"].replace({2: 0})

In [ ]:
# Replace values of "Y" with 1 in the "RenalDiseaseIndicator" column

train["RenalDiseaseIndicator"] = train["RenalDiseaseIndicator"].replace({"Y": 1})

In [ ]:
# Convert "DOB" and "DOD" columns to datetime

birthdate = pd.to_datetime(train["DOB"])
enddate = pd.to_datetime(train["DOD"])

# Check whether the patient is dead or alive
def alive_function(value):
    if value == True:
        return 1
    else:
        return 0

alive = pd.isna(enddate).apply(alive_function)

# Get the age of the patient

max_date = enddate.dropna().max()
enddate[pd.isna(enddate)] = max_date
period = (((enddate - birthdate).dt.days / 365).astype(int))

# Assign age and alive status to the train DataFrame

train["Age"] = period
train["Alive"] = alive

In [ ]:
# dropping DOD, DOB from train

train=train.drop(columns=["DOB", "DOD"], axis=1)

In [ ]:
# Extracting a similarity feature out of Physician type

def check_physician_similarity(data_row):
    attending_operating = data_row["AttendingPhysician"] == data_row["OperatingPhysician"]
    operating_other = data_row["OperatingPhysician"] == data_row["OtherPhysician"]
    attending_other = data_row["AttendingPhysician"] == data_row["OtherPhysician"]

    if attending_operating and operating_other:  # attending = operating = other
        return 0
    elif attending_operating and not operating_other:  # attending = operating != other
        return 1
    elif not attending_operating and operating_other:  # attending != operating = other
        return 2
    else:  # attending != operating != other
        return 3

# Apply the function to each row of the DataFrame
train['Same_Physician'] = train.apply(check_physician_similarity, axis=1)

# Define a function to count the number of non-null physician IDs
def physician_count(row, list_count):
    count = 0
    for col in list_count:
        if not pd.isnull(row[col]):
            count += 1
    return count

list_count = ["AttendingPhysician", "OperatingPhysician", "OtherPhysician"]

train['Physician_Count'] = train.apply(physician_count, axis=1, args=(list_count,))

In [ ]:
cont_var=["InscClaimAmtReimbursed","DeductibleAmtPaid","NoOfMonths_PartACov","NoOfMonths_PartBCov",
           "IPAnnualReimbursementAmt","IPAnnualDeductibleAmt","OPAnnualReimbursementAmt","OPAnnualDeductibleAmt"]

In [ ]:
def groupby(df,by,vars_to_group,methods,col_ident,as_index=True,agg=False):
    if agg:
        grouped=df.groupby(by=by,as_index=as_index)[vars_to_group].agg(methods)
        cols=['_'.join(col) for col in grouped.columns.values]
        cols=[col_ident+"_"+col for col in cols]
        grouped.columns=cols
        return grouped

    else:
        concat=df.groupby(by=by,as_index=as_index)[vars_to_group].transform(methods[0])
        cols=[ col_ident+"_"+col+"_"+methods[0] for col in concat.columns ]
        concat.columns=cols

        for method in methods[1:]:
            grouped=df.groupby(by=by,as_index=as_index)[vars_to_group].transform(method)
            cols=[col_ident+"_"+col+"_"+method for col in grouped.columns]
            grouped.columns=cols
            concat=pd.concat([concat,grouped],axis=1)

        return concat

In [ ]:
traincopy=train

In [ ]:
feat_provider=groupby(traincopy,["Provider"],cont_var,["mean","std"],"Provider",
                       True,False)

In [ ]:
# Similarly producing 3 more sets of new features by accumulating existing features

feat_bene=groupby(traincopy,["BeneID"],cont_var,["mean","std"],"Bene",
                       True,False)

feat_diagcode=groupby(traincopy,["ClmDiagnosisCode_1"],cont_var,["mean","std"],"DiagCode",
                       True,False)

feat_proccode=groupby(traincopy,["ClmProcedureCode_1"],cont_var,["mean","std"],"ProcCode",
                       True,False)


In [ ]:
prev_feat=["Same_Physician","Physician_Count","Claim_Period","Admit_Period","Age","Alive","Provider"]
prevfeat=traincopy[prev_feat]

In [ ]:
agg_data=pd.concat([prevfeat,feat_provider,feat_bene,feat_diagcode,feat_proccode],axis=1)

In [ ]:
grouped=agg_data.groupby(by=["Provider"]).agg("mean").reset_index()

In [ ]:
grouped=grouped.fillna(0)

features=grouped.iloc[:,1:]


In [ ]:
# Using Z-Score Normalization for features

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
featuresstand=scaler.fit_transform(features)

In [ ]:
x_test=features
x_test.shape

(1353, 70)

# Model Inference from Saved XGBoost model

In [ ]:
import joblib

# Load the saved model from file
loaded_model = joblib.load('/kaggle/input/xgboostsaved/other/xgboost_saved/1/xgb_model.pkl')

# Perform inference using the loaded model
y_pred = loaded_model.predict(x_test)


In [ ]:
y_pred

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
len(y_pred)

1353

In [ ]:
y_pred_int = y_pred.astype(int)

# Save the integer array to a CSV file
np.savetxt("predictions_on_test.csv", y_pred_int, delimiter=",")